### Imports

In [1]:
import os, time, pickle, random, time
from datetime import datetime
import numpy as np
from time import localtime, strftime
import logging, scipy
import tensorflow as tf
import tensorlayer as tl
from model import SRGAN_g, SRGAN_d, Vgg19_simple_api
from utils import *
from config import config, log_config
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image
import random

### Hyper Params

In [2]:
## Adam
batch_size = config.TRAIN.batch_size
lr_init = config.TRAIN.lr_init
beta1 = config.TRAIN.beta1
## initialize G
n_epoch_init = 5
## adversarial learning (SRGAN)
n_epoch = 1000
lr_decay = config.TRAIN.lr_decay
decay_every = config.TRAIN.decay_every
ni = int(np.sqrt(batch_size))

### Dirs and Stuff

In [3]:
## create folders to save result images and trained model
save_dir_ginit = "samples/{}_ginit".format("srgan")
save_dir_gan = "samples/{}_gan".format("srgan")
tl.files.exists_or_mkdir(save_dir_ginit)
tl.files.exists_or_mkdir(save_dir_gan)
checkpoint_dir = "checkpoint"  # checkpoint_resize_conv
tl.files.exists_or_mkdir(checkpoint_dir)

[TL] [!] samples/srgan_ginit exists ...
[TL] [!] samples/srgan_gan exists ...
[TL] [!] checkpoint exists ...


True

### Data

In [4]:
train_hr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.hr_img_path, regx='.*.png', printable=False))
train_lr_img_list = sorted(tl.files.load_file_list(path=config.TRAIN.lr_img_path, regx='.*.png', printable=False))
valid_hr_img_list = sorted(tl.files.load_file_list(path=config.VALID.hr_img_path, regx='.*.png', printable=False))
valid_lr_img_list = sorted(tl.files.load_file_list(path=config.VALID.lr_img_path, regx='.*.png', printable=False))

In [5]:
train_hr_imgs = tl.vis.read_images(train_hr_img_list, path=config.TRAIN.hr_img_path, n_threads=32)
train_lr_img = tl.vis.read_images(train_lr_img_list, path=config.TRAIN.lr_img_path, n_threads=32)
valid_hr_img = tl.vis.read_images(valid_hr_img_list, path=config.VALID.hr_img_path, n_threads=32)
valid_lr_img = tl.vis.read_images(valid_lr_img_list, path=config.VALID.lr_img_path, n_threads=32)

[TL] read 32 from mainDataset/train/HR
[TL] read 64 from mainDataset/train/HR
[TL] read 96 from mainDataset/train/HR
[TL] read 128 from mainDataset/train/HR
[TL] read 160 from mainDataset/train/HR
[TL] read 192 from mainDataset/train/HR
[TL] read 224 from mainDataset/train/HR
[TL] read 256 from mainDataset/train/HR
[TL] read 288 from mainDataset/train/HR
[TL] read 320 from mainDataset/train/HR
[TL] read 352 from mainDataset/train/HR
[TL] read 384 from mainDataset/train/HR
[TL] read 416 from mainDataset/train/HR
[TL] read 448 from mainDataset/train/HR
[TL] read 480 from mainDataset/train/HR
[TL] read 512 from mainDataset/train/HR
[TL] read 544 from mainDataset/train/HR
[TL] read 576 from mainDataset/train/HR
[TL] read 608 from mainDataset/train/HR
[TL] read 640 from mainDataset/train/HR
[TL] read 672 from mainDataset/train/HR
[TL] read 704 from mainDataset/train/HR
[TL] read 736 from mainDataset/train/HR
[TL] read 768 from mainDataset/train/HR
[TL] read 800 from mainDataset/train/HR
[TL

[TL] read 288 from mainDataset/dev/LR
[TL] read 320 from mainDataset/dev/LR
[TL] read 352 from mainDataset/dev/LR
[TL] read 365 from mainDataset/dev/LR


In [6]:
for dataset in [train_hr_imgs,train_lr_img,valid_hr_img,valid_lr_img]:
    for i,img in enumerate(dataset):
        if len(img.shape) == 2:
            print("B&W")

In [9]:
'''# Removing B&W images since it causes problems in Channel
for i,img in enumerate(train_hr_imgs):
    if len(img.shape) == 2:
        train_hr_imgs.pop(i)
for i,img in enumerate(train_lr_img):
    if len(img.shape) == 2:
        train_hr_imgs.pop(i)
for i,img in enumerate(valid_hr_img):
    if len(img.shape) == 2:
        train_hr_imgs.pop(i)
for i,img in enumerate(valid_lr_img):
    if len(img.shape) == 2:
        train_hr_imgs.pop(i)   '''

'# Removing B&W images since it causes problems in Channel\nfor i,img in enumerate(train_hr_imgs):\n    if len(img.shape) == 2:\n        train_hr_imgs.pop(i)\nfor i,img in enumerate(train_lr_img):\n    if len(img.shape) == 2:\n        train_hr_imgs.pop(i)\nfor i,img in enumerate(valid_hr_img):\n    if len(img.shape) == 2:\n        train_hr_imgs.pop(i)\nfor i,img in enumerate(valid_lr_img):\n    if len(img.shape) == 2:\n        train_hr_imgs.pop(i)   '

In [10]:
'''cropDim = 5000
for img in train_hr_imgs:
    minDim = min(img.shape[0:2])
    if cropDim > minDim :
        cropDim = minDim'''

'cropDim = 5000\nfor img in train_hr_imgs:\n    minDim = min(img.shape[0:2])\n    if cropDim > minDim :\n        cropDim = minDim'

In [11]:
'''cropDim -= 1+(cropDim-1)%4'''

'cropDim -= 1+(cropDim-1)%4'

In [7]:
cropDim=224

In [8]:
sample_imgs = random.sample(train_hr_imgs, batch_size)
#sample_imgs = tl.vis.read_images(train_hr_img_list[0:batch_size], path=config.TRAIN.hr_img_path, n_threads=32) # if no pre-load train set
sample_imgs_224 = tl.prepro.threading_data(sample_imgs, fn=crop_sub_imgs_fn,dim=cropDim,is_random=False)
print('sample HR sub-image:', sample_imgs_224.shape, sample_imgs_224.min(), sample_imgs_224.max())
sample_imgs_56 = tl.prepro.threading_data(sample_imgs_224,dim=int(cropDim/4),fn=downsample_fn)
print('sample LR sub-image:', sample_imgs_56.shape, sample_imgs_56.min(), sample_imgs_56.max())
tl.vis.save_images(sample_imgs_56, [ni, ni], save_dir_ginit + '/_train_sample_56.png')
tl.vis.save_images(sample_imgs_224, [ni, ni], save_dir_ginit + '/_train_sample_224.png')
tl.vis.save_images(sample_imgs_56, [ni, ni], save_dir_gan + '/_train_sample_56.png')
tl.vis.save_images(sample_imgs_224, [ni, ni], save_dir_gan + '/_train_sample_224.png')

sample HR sub-image: (25, 224, 224, 3) -1.0 1.0
sample LR sub-image: (25, 56, 56, 3) -1.0 1.0


In [9]:
t_image = tf.placeholder('float32', [batch_size, int(cropDim/4), int(cropDim/4), 3], name='t_image_input_to_SRGAN_generator')

In [10]:
t_target_image = tf.placeholder('float32', [batch_size, cropDim, cropDim, 3], name='t_target_image')

In [11]:
net_g = SRGAN_g(t_image, is_train=True, reuse=False)

[TL] InputLayer  SRGAN_g/in: (25, 56, 56, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
Instructions for updating:
Colocations handled automatically by placer.
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (25, 56, 56, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: True
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strid

[TL] Conv2d SRGAN_g/n64s1/c2/15: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/15: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/b_residual_add/15: size: (25, 56, 56, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: True
[TL] ElementwiseLayer SRGAN_g/add3: size: (25, 56, 56, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out: n_fi

In [12]:
net_d, logits_real = SRGAN_d(t_target_image, is_train=True, reuse=False)

[TL] WARNING: Function: `tensorlayer.layers.utils.set_name_reuse` (in file: /home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/tensorlayer/layers/utils.py) is deprecated and will be removed after 2018-06-30.
Instructions for updating: TensorLayer relies on TensorFlow to check name reusing

[TL] WARNING: this method is DEPRECATED and has no effect, please remove it from your code.
[TL] InputLayer  SRGAN_d/input/images: (25, 224, 224, 3)
[TL] Conv2d SRGAN_d/h0/c: n_filter: 64 filter_size: (4, 4) strides: (2, 2) pad: SAME act: <lambda>
[TL] WARNING: Function: `tensorlayer.activation.leaky_relu` (in file: /home/ubuntu/anaconda3/envs/srgan/lib/python3.7/site-packages/tensorlayer/activation.py) is deprecated and will be removed after 2018-09-30.
Instructions for updating: This API is deprecated. Please use as `tf.nn.leaky_relu`

[TL] Conv2d SRGAN_d/h1/c: n_filter: 128 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h1/bn: decay: 0.90000

In [13]:
_, logits_fake = SRGAN_d(net_g.outputs, is_train=True, reuse=True)

[TL] WARNING: this method is DEPRECATED and has no effect, please remove it from your code.
[TL] InputLayer  SRGAN_d/input/images: (25, 224, 224, 3)
[TL] Conv2d SRGAN_d/h0/c: n_filter: 64 filter_size: (4, 4) strides: (2, 2) pad: SAME act: <lambda>
[TL] Conv2d SRGAN_d/h1/c: n_filter: 128 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h1/bn: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: True
[TL] Conv2d SRGAN_d/h2/c: n_filter: 256 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h2/bn: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: True
[TL] Conv2d SRGAN_d/h3/c: n_filter: 512 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h3/bn: decay: 0.900000 epsilon: 0.000010 act: <lambda> is_train: True
[TL] Conv2d SRGAN_d/h4/c: n_filter: 1024 filter_size: (4, 4) strides: (2, 2) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_d/h4/bn: de

In [14]:
net_g.print_params(False)
net_g.print_layers()
net_d.print_params(False)
net_d.print_layers()

[TL]   param   0: SRGAN_g/n64s1/c/kernel:0 (3, 3, 3, 64)      float32_ref
[TL]   param   1: SRGAN_g/n64s1/c/bias:0 (64,)              float32_ref
[TL]   param   2: SRGAN_g/n64s1/c1/0/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param   3: SRGAN_g/n64s1/b1/0/beta:0 (64,)              float32_ref
[TL]   param   4: SRGAN_g/n64s1/b1/0/gamma:0 (64,)              float32_ref
[TL]   param   5: SRGAN_g/n64s1/b1/0/moving_mean:0 (64,)              float32_ref
[TL]   param   6: SRGAN_g/n64s1/b1/0/moving_variance:0 (64,)              float32_ref
[TL]   param   7: SRGAN_g/n64s1/c2/0/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param   8: SRGAN_g/n64s1/b2/0/beta:0 (64,)              float32_ref
[TL]   param   9: SRGAN_g/n64s1/b2/0/gamma:0 (64,)              float32_ref
[TL]   param  10: SRGAN_g/n64s1/b2/0/moving_mean:0 (64,)              float32_ref
[TL]   param  11: SRGAN_g/n64s1/b2/0/moving_variance:0 (64,)              float32_ref
[TL]   param  12: SRGAN_g/n64s1/c1/1/kernel:0 (3, 3, 64, 64)  

[TL]   param 104: SRGAN_g/n64s1/b1/10/gamma:0 (64,)              float32_ref
[TL]   param 105: SRGAN_g/n64s1/b1/10/moving_mean:0 (64,)              float32_ref
[TL]   param 106: SRGAN_g/n64s1/b1/10/moving_variance:0 (64,)              float32_ref
[TL]   param 107: SRGAN_g/n64s1/c2/10/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param 108: SRGAN_g/n64s1/b2/10/beta:0 (64,)              float32_ref
[TL]   param 109: SRGAN_g/n64s1/b2/10/gamma:0 (64,)              float32_ref
[TL]   param 110: SRGAN_g/n64s1/b2/10/moving_mean:0 (64,)              float32_ref
[TL]   param 111: SRGAN_g/n64s1/b2/10/moving_variance:0 (64,)              float32_ref
[TL]   param 112: SRGAN_g/n64s1/c1/11/kernel:0 (3, 3, 64, 64)     float32_ref
[TL]   param 113: SRGAN_g/n64s1/b1/11/beta:0 (64,)              float32_ref
[TL]   param 114: SRGAN_g/n64s1/b1/11/gamma:0 (64,)              float32_ref
[TL]   param 115: SRGAN_g/n64s1/b1/11/moving_mean:0 (64,)              float32_ref
[TL]   param 116: SRGAN_g/n64s1/b1/11/

[TL]   layer  36: SRGAN_g/b_residual_add/6:0 (25, 56, 56, 64)    float32
[TL]   layer  37: SRGAN_g/n64s1/c1/7/Conv2D:0 (25, 56, 56, 64)    float32
[TL]   layer  38: SRGAN_g/n64s1/b1/7/Relu:0 (25, 56, 56, 64)    float32
[TL]   layer  39: SRGAN_g/n64s1/c2/7/Conv2D:0 (25, 56, 56, 64)    float32
[TL]   layer  40: SRGAN_g/n64s1/b2/7/batchnorm/Add_1:0 (25, 56, 56, 64)    float32
[TL]   layer  41: SRGAN_g/b_residual_add/7:0 (25, 56, 56, 64)    float32
[TL]   layer  42: SRGAN_g/n64s1/c1/8/Conv2D:0 (25, 56, 56, 64)    float32
[TL]   layer  43: SRGAN_g/n64s1/b1/8/Relu:0 (25, 56, 56, 64)    float32
[TL]   layer  44: SRGAN_g/n64s1/c2/8/Conv2D:0 (25, 56, 56, 64)    float32
[TL]   layer  45: SRGAN_g/n64s1/b2/8/batchnorm/Add_1:0 (25, 56, 56, 64)    float32
[TL]   layer  46: SRGAN_g/b_residual_add/8:0 (25, 56, 56, 64)    float32
[TL]   layer  47: SRGAN_g/n64s1/c1/9/Conv2D:0 (25, 56, 56, 64)    float32
[TL]   layer  48: SRGAN_g/n64s1/b1/9/Relu:0 (25, 56, 56, 64)    float32
[TL]   layer  49: SRGAN_g/n64

[TL]   layer   1: SRGAN_d/h0/c/leaky_relu:0 (25, 112, 112, 64)    float32
[TL]   layer   2: SRGAN_d/h1/c/Conv2D:0 (25, 56, 56, 128)    float32
[TL]   layer   3: SRGAN_d/h1/bn/leaky_relu:0 (25, 56, 56, 128)    float32
[TL]   layer   4: SRGAN_d/h2/c/Conv2D:0 (25, 28, 28, 256)    float32
[TL]   layer   5: SRGAN_d/h2/bn/leaky_relu:0 (25, 28, 28, 256)    float32
[TL]   layer   6: SRGAN_d/h3/c/Conv2D:0 (25, 14, 14, 512)    float32
[TL]   layer   7: SRGAN_d/h3/bn/leaky_relu:0 (25, 14, 14, 512)    float32
[TL]   layer   8: SRGAN_d/h4/c/Conv2D:0 (25, 7, 7, 1024)    float32
[TL]   layer   9: SRGAN_d/h4/bn/leaky_relu:0 (25, 7, 7, 1024)    float32
[TL]   layer  10: SRGAN_d/h5/c/Conv2D:0 (25, 4, 4, 2048)    float32
[TL]   layer  11: SRGAN_d/h5/bn/leaky_relu:0 (25, 4, 4, 2048)    float32
[TL]   layer  12: SRGAN_d/h6/c/Conv2D:0 (25, 4, 4, 1024)    float32
[TL]   layer  13: SRGAN_d/h6/bn/leaky_relu:0 (25, 4, 4, 1024)    float32
[TL]   layer  14: SRGAN_d/h7/c/Conv2D:0 (25, 4, 4, 512)    float32
[TL]   

In [15]:
t_target_image_224 = tf.image.resize_images(t_target_image, size=[224, 224], method=0,align_corners=False)

In [16]:
t_predict_image_224 = tf.image.resize_images(net_g.outputs, size=[224, 224], method=0, align_corners=False)

In [17]:
net_vgg, vgg_target_emb = Vgg19_simple_api((t_target_image_224 + 1) / 2, reuse=False)
_, vgg_predict_emb = Vgg19_simple_api((t_predict_image_224 + 1) / 2, reuse=True)

build model started
[TL] InputLayer  VGG19/input: (25, 224, 224, 3)
[TL] Conv2d VGG19/conv1_1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv1_2: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d VGG19/pool1: filter_size: (2, 2) strides: (2, 2) padding: SAME
Instructions for updating:
Use keras.layers.max_pooling2d instead.
[TL] Conv2d VGG19/conv2_1: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv2_2: n_filter: 128 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] MaxPool2d VGG19/pool2: filter_size: (2, 2) strides: (2, 2) padding: SAME
[TL] Conv2d VGG19/conv3_1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv3_2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv3_3: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d VGG19/conv3_4: n_fil

In [18]:
## test inference
net_g_test = SRGAN_g(t_image, is_train=False, reuse=True)

[TL] InputLayer  SRGAN_g/in: (25, 56, 56, 3)
[TL] Conv2d SRGAN_g/n64s1/c: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: relu
[TL] Conv2d SRGAN_g/n64s1/c1/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/0: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/0: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b2/0: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/b_residual_add/0: size: (25, 56, 56, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c1/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b1/1: decay: 0.900000 epsilon: 0.000010 act: relu is_train: False
[TL] Conv2d SRGAN_g/n64s1/c2/1: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n

[TL] ElementwiseLayer SRGAN_g/b_residual_add/15: size: (25, 56, 56, 64) fn: add
[TL] Conv2d SRGAN_g/n64s1/c/m: n_filter: 64 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] BatchNormLayer SRGAN_g/n64s1/b/m: decay: 0.900000 epsilon: 0.000010 act: No Activation is_train: False
[TL] ElementwiseLayer SRGAN_g/add3: size: (25, 56, 56, 64) fn: add
[TL] Conv2d SRGAN_g/n256s1/1: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/1: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/n256s1/2: n_filter: 256 filter_size: (3, 3) strides: (1, 1) pad: SAME act: No Activation
[TL] SubpixelConv2d  SRGAN_g/pixelshufflerx2/2: scale: 2 n_out_channel: 64 act: relu
[TL] Conv2d SRGAN_g/out: n_filter: 3 filter_size: (1, 1) strides: (1, 1) pad: SAME act: tanh


In [19]:
# ###========================== DEFINE TRAIN OPS ==========================###
d_loss1 = tl.cost.sigmoid_cross_entropy(logits_real, tf.ones_like(logits_real), name='d1')
d_loss2 = tl.cost.sigmoid_cross_entropy(logits_fake, tf.zeros_like(logits_fake), name='d2')
d_loss = d_loss1 + d_loss2

g_gan_loss = 1e-3 * tl.cost.sigmoid_cross_entropy(logits_fake, tf.ones_like(logits_fake), name='g')
mse_loss = tl.cost.mean_squared_error(net_g.outputs, t_target_image, is_mean=True)
vgg_loss = 2e-6 * tl.cost.mean_squared_error(vgg_predict_emb.outputs, vgg_target_emb.outputs, is_mean=True)

g_loss = mse_loss + vgg_loss + g_gan_loss

g_vars = tl.layers.get_variables_with_name('SRGAN_g', True, True)
d_vars = tl.layers.get_variables_with_name('SRGAN_d', True, True)

with tf.variable_scope('learning_rate'):
    lr_v = tf.Variable(lr_init, trainable=False)

[TL]   [*] geting variables with SRGAN_g
[TL]   got   0: SRGAN_g/n64s1/c/kernel:0   (3, 3, 3, 64)
[TL]   got   1: SRGAN_g/n64s1/c/bias:0   (64,)
[TL]   got   2: SRGAN_g/n64s1/c1/0/kernel:0   (3, 3, 64, 64)
[TL]   got   3: SRGAN_g/n64s1/b1/0/beta:0   (64,)
[TL]   got   4: SRGAN_g/n64s1/b1/0/gamma:0   (64,)
[TL]   got   5: SRGAN_g/n64s1/c2/0/kernel:0   (3, 3, 64, 64)
[TL]   got   6: SRGAN_g/n64s1/b2/0/beta:0   (64,)
[TL]   got   7: SRGAN_g/n64s1/b2/0/gamma:0   (64,)
[TL]   got   8: SRGAN_g/n64s1/c1/1/kernel:0   (3, 3, 64, 64)
[TL]   got   9: SRGAN_g/n64s1/b1/1/beta:0   (64,)
[TL]   got  10: SRGAN_g/n64s1/b1/1/gamma:0   (64,)
[TL]   got  11: SRGAN_g/n64s1/c2/1/kernel:0   (3, 3, 64, 64)
[TL]   got  12: SRGAN_g/n64s1/b2/1/beta:0   (64,)
[TL]   got  13: SRGAN_g/n64s1/b2/1/gamma:0   (64,)
[TL]   got  14: SRGAN_g/n64s1/c1/2/kernel:0   (3, 3, 64, 64)
[TL]   got  15: SRGAN_g/n64s1/b1/2/beta:0   (64,)
[TL]   got  16: SRGAN_g/n64s1/b1/2/gamma:0   (64,)
[TL]   got  17: SRGAN_g/n64s1/c2/2/kernel:0  

In [20]:
g_optim_init = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(mse_loss, var_list=g_vars)
## SRGAN
g_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(g_loss, var_list=g_vars)
d_optim = tf.train.AdamOptimizer(lr_v, beta1=beta1).minimize(d_loss, var_list=d_vars)

Instructions for updating:
Use tf.cast instead.


In [23]:
###========================== RESTORE MODEL =============================###
sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=False))
tl.layers.initialize_global_variables(sess)
if tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/g_{}.npz'.format("srgan"), network=net_g) is False:
    tl.files.load_and_assign_npz(sess=sess, name=checkpoint_dir + '/g_{}_init.npz'.format("srgan"), network=net_g)

[TL] ERROR: file checkpoint/g_srgan.npz doesn't exist.


In [24]:
###============================= LOAD VGG ===============================###
vgg19_npy_path = "../vgg19.npy"
if not os.path.isfile(vgg19_npy_path):
    print("Please download vgg19.npz from : https://github.com/machrisaa/tensorflow-vgg")
    exit()
npz = np.load(vgg19_npy_path, encoding='latin1').item()

params = []
for val in sorted(npz.items()):
    W = np.asarray(val[1][0])
    b = np.asarray(val[1][1])
    print("  Loading %s: %s, %s" % (val[0], W.shape, b.shape))
    params.extend([W, b])
tl.files.assign_params(sess, params, net_vgg)
# net_vgg.print_params(False)
# net_vgg.print_layers()

  Loading conv1_1: (3, 3, 3, 64), (64,)
  Loading conv1_2: (3, 3, 64, 64), (64,)
  Loading conv2_1: (3, 3, 64, 128), (128,)
  Loading conv2_2: (3, 3, 128, 128), (128,)
  Loading conv3_1: (3, 3, 128, 256), (256,)
  Loading conv3_2: (3, 3, 256, 256), (256,)
  Loading conv3_3: (3, 3, 256, 256), (256,)
  Loading conv3_4: (3, 3, 256, 256), (256,)
  Loading conv4_1: (3, 3, 256, 512), (512,)
  Loading conv4_2: (3, 3, 512, 512), (512,)
  Loading conv4_3: (3, 3, 512, 512), (512,)
  Loading conv4_4: (3, 3, 512, 512), (512,)
  Loading conv5_1: (3, 3, 512, 512), (512,)
  Loading conv5_2: (3, 3, 512, 512), (512,)
  Loading conv5_3: (3, 3, 512, 512), (512,)
  Loading conv5_4: (3, 3, 512, 512), (512,)
  Loading fc6: (25088, 4096), (4096,)
  Loading fc7: (4096, 4096), (4096,)
  Loading fc8: (4096, 1000), (1000,)


[<tf.Tensor 'Assign_173:0' shape=(3, 3, 3, 64) dtype=float32_ref>,
 <tf.Tensor 'Assign_174:0' shape=(64,) dtype=float32_ref>,
 <tf.Tensor 'Assign_175:0' shape=(3, 3, 64, 64) dtype=float32_ref>,
 <tf.Tensor 'Assign_176:0' shape=(64,) dtype=float32_ref>,
 <tf.Tensor 'Assign_177:0' shape=(3, 3, 64, 128) dtype=float32_ref>,
 <tf.Tensor 'Assign_178:0' shape=(128,) dtype=float32_ref>,
 <tf.Tensor 'Assign_179:0' shape=(3, 3, 128, 128) dtype=float32_ref>,
 <tf.Tensor 'Assign_180:0' shape=(128,) dtype=float32_ref>,
 <tf.Tensor 'Assign_181:0' shape=(3, 3, 128, 256) dtype=float32_ref>,
 <tf.Tensor 'Assign_182:0' shape=(256,) dtype=float32_ref>,
 <tf.Tensor 'Assign_183:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Tensor 'Assign_184:0' shape=(256,) dtype=float32_ref>,
 <tf.Tensor 'Assign_185:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Tensor 'Assign_186:0' shape=(256,) dtype=float32_ref>,
 <tf.Tensor 'Assign_187:0' shape=(3, 3, 256, 256) dtype=float32_ref>,
 <tf.Tensor 'Assign_188:0' s

In [25]:
def train_hr_imgs_datagen(train_hr_imgs,batch_size):
    while True:
        idx = np.random.randint(0,len(train_hr_imgs),batch_size)
        yield [train_hr_imgs[x] for x in idx]

In [29]:
sess.run(tf.assign(lr_v, lr_init))
print(" ** fixed learning rate: %f (for init G)" % lr_init)
datagen = train_hr_imgs_datagen(train_hr_imgs,batch_size)
train_writer = tf.summary.FileWriter( './logs/1/train ', sess.graph)
for epoch in range(0, n_epoch_init + 1):
    epoch_time = time.time()
    total_mse_loss, n_iter = 0, 0

    ## If your machine cannot load all images into memory, you should use
    ## this one to load batch of images while training.
    # random.shuffle(train_hr_img_list)
    # for idx in range(0, len(train_hr_img_list), batch_size):
    #     step_time = time.time()
    #     b_imgs_list = train_hr_img_list[idx : idx + batch_size]
    #     b_imgs = tl.prepro.threading_data(b_imgs_list, fn=get_imgs_fn, path=config.TRAIN.hr_img_path)
    #     b_imgs_384 = tl.prepro.threading_data(b_imgs, fn=crop_sub_imgs_fn, is_random=True)
    #     b_imgs_96 = tl.prepro.threading_data(b_imgs_384, fn=downsample_fn)

    ## If your machine have enough memory, please pre-load the whole train set.
    random.shuffle(train_hr_img_list)
    for idx in range(0, len(train_hr_imgs), batch_size):
        step_time = time.time()        
        #b_imgs_384 = tl.prepro.threading_data(train_hr_imgs, fn=crop_sub_imgs_fn, dim=cropDim, is_random=True)
        #b_imgs_96 = tl.prepro.threading_data(train_hr_imgs, fn=downsample_fn,dim=int(cropDim/4))
        b_imgs_384 = tl.prepro.threading_data(next(datagen), fn=crop_sub_imgs_fn,dim=cropDim, is_random=True)
        b_imgs_96 = tl.prepro.threading_data(b_imgs_384, fn=downsample_fn,dim=int(cropDim/4))
        ## update G
        errM, _ = sess.run([mse_loss, g_optim_init], {t_image: b_imgs_96, t_target_image: b_imgs_384})
        
        print("Epoch [%2d/%2d] %4d time: %4.4fs, mse: %.8f " % (epoch, n_epoch_init, n_iter, time.time() - step_time, errM))
        total_mse_loss += errM
        n_iter += 1
    log = "[*] Epoch: [%2d/%2d] time: %4.4fs, mse: %.8f" % (epoch, n_epoch_init, time.time() - epoch_time, total_mse_loss / n_iter)
    print(log)

    ## quick evaluation on train set
    if (epoch != 0) and (epoch % 1 == 0):
        out = sess.run(net_g_test.outputs, {t_image: sample_imgs_56})  #; print('gen sub-image:', out.shape, out.min(), out.max())
        print("[*] save images")
        tl.vis.save_images(out, [ni, ni], save_dir_ginit + '/train_%d.png' % epoch)

    ## save model
    if (epoch != 0) and (epoch % 10 == 0):
        tl.files.save_npz(net_g.all_params, name=checkpoint_dir + '/g_{}_init.npz'.format("train"), sess=sess)


 ** fixed learning rate: 0.000100 (for init G)
Epoch [ 0/ 5]    0 time: 1.4665s, mse: 0.36366630 
Epoch [ 0/ 5]    1 time: 0.8000s, mse: 0.31461895 
Epoch [ 0/ 5]    2 time: 0.7971s, mse: 0.28192452 
Epoch [ 0/ 5]    3 time: 0.7970s, mse: 0.27121753 
Epoch [ 0/ 5]    4 time: 0.7973s, mse: 0.25984412 
Epoch [ 0/ 5]    5 time: 0.8003s, mse: 0.28849080 
Epoch [ 0/ 5]    6 time: 0.7950s, mse: 0.24017832 
Epoch [ 0/ 5]    7 time: 0.7959s, mse: 0.20039183 
Epoch [ 0/ 5]    8 time: 0.7948s, mse: 0.26194808 
Epoch [ 0/ 5]    9 time: 0.7932s, mse: 0.25581202 
Epoch [ 0/ 5]   10 time: 0.7984s, mse: 0.25892043 
Epoch [ 0/ 5]   11 time: 0.7973s, mse: 0.24927999 
Epoch [ 0/ 5]   12 time: 0.7949s, mse: 0.22902279 
Epoch [ 0/ 5]   13 time: 0.7938s, mse: 0.26392478 
Epoch [ 0/ 5]   14 time: 0.7956s, mse: 0.23937926 
Epoch [ 0/ 5]   15 time: 0.7982s, mse: 0.27780476 
Epoch [ 0/ 5]   16 time: 0.7968s, mse: 0.17166419 
Epoch [ 0/ 5]   17 time: 0.7954s, mse: 0.22285339 
Epoch [ 0/ 5]   18 time: 0.7943s, m

Epoch [ 1/ 5]   42 time: 0.8037s, mse: 0.05890818 
Epoch [ 1/ 5]   43 time: 0.8062s, mse: 0.06956996 
Epoch [ 1/ 5]   44 time: 0.8054s, mse: 0.06413866 
Epoch [ 1/ 5]   45 time: 0.8070s, mse: 0.06104607 
Epoch [ 1/ 5]   46 time: 0.7996s, mse: 0.06452899 
Epoch [ 1/ 5]   47 time: 0.8043s, mse: 0.05824335 
Epoch [ 1/ 5]   48 time: 0.8029s, mse: 0.05893613 
Epoch [ 1/ 5]   49 time: 0.8056s, mse: 0.05341320 
Epoch [ 1/ 5]   50 time: 0.8017s, mse: 0.06981802 
Epoch [ 1/ 5]   51 time: 0.8014s, mse: 0.06602208 
Epoch [ 1/ 5]   52 time: 0.8020s, mse: 0.07746319 
Epoch [ 1/ 5]   53 time: 0.8087s, mse: 0.04932456 
Epoch [ 1/ 5]   54 time: 0.8040s, mse: 0.07895897 
Epoch [ 1/ 5]   55 time: 0.8016s, mse: 0.07344941 
Epoch [ 1/ 5]   56 time: 0.8059s, mse: 0.09832211 
Epoch [ 1/ 5]   57 time: 0.8042s, mse: 0.05498318 
Epoch [ 1/ 5]   58 time: 0.8045s, mse: 0.05502464 
Epoch [ 1/ 5]   59 time: 0.8022s, mse: 0.06572552 
Epoch [ 1/ 5]   60 time: 0.8018s, mse: 0.04621403 
Epoch [ 1/ 5]   61 time: 0.8027

KeyboardInterrupt: 

In [45]:
###========================= train GAN (SRGAN) =========================###
for epoch in range(0, n_epoch + 1):
    ## update learning rate
    if epoch != 0 and (epoch % decay_every == 0):
        new_lr_decay = lr_decay**(epoch // decay_every)
        sess.run(tf.assign(lr_v, lr_init * new_lr_decay))
        log = " ** new learning rate: %f (for GAN)" % (lr_init * new_lr_decay)
        print(log)
    elif epoch == 0:
        sess.run(tf.assign(lr_v, lr_init))
        log = " ** init lr: %f  decay_every_init: %d, lr_decay: %f (for GAN)" % (lr_init, decay_every, lr_decay)
        print(log)

    epoch_time = time.time()
    total_d_loss, total_g_loss, n_iter = 0, 0, 0

    ## If your machine cannot load all images into memory, you should use
    ## this one to load batch of images while training.
    # random.shuffle(train_hr_img_list)
    # for idx in range(0, len(train_hr_img_list), batch_size):
    #     step_time = time.time()
    #     b_imgs_list = train_hr_img_list[idx : idx + batch_size]
    #     b_imgs = tl.prepro.threading_data(b_imgs_list, fn=get_imgs_fn, path=config.TRAIN.hr_img_path)
    #     b_imgs_384 = tl.prepro.threading_data(b_imgs, fn=crop_sub_imgs_fn, is_random=True)
    #     b_imgs_96 = tl.prepro.threading_data(b_imgs_384, fn=downsample_fn)

    ## If your machine have enough memory, please pre-load the whole train set.
    for idx in range(0, len(train_hr_imgs), batch_size):
        step_time = time.time()
        b_imgs_384 = tl.prepro.threading_data(next(datagen), fn=crop_sub_imgs_fn,dim = cropDim, is_random=True)
        b_imgs_96 = tl.prepro.threading_data(b_imgs_384, fn=downsample_fn,dim=int(cropDim/4))
        ## update D
        errD, _ = sess.run([d_loss, d_optim], {t_image: b_imgs_96, t_target_image: b_imgs_384})
        ## update G
        errG, errM, errV, errA, _ = sess.run([g_loss, mse_loss, vgg_loss, g_gan_loss, g_optim], {t_image: b_imgs_96, t_target_image: b_imgs_384})
        print("Epoch [%2d/%2d] %4d time: %4.4fs, d_loss: %.8f g_loss: %.8f (mse: %.6f vgg: %.6f adv: %.6f)" %
              (epoch, n_epoch, n_iter, time.time() - step_time, errD, errG, errM, errV, errA))
        total_d_loss += errD
        total_g_loss += errG
        n_iter += 1

    log = "[*] Epoch: [%2d/%2d] time: %4.4fs, d_loss: %.8f g_loss: %.8f" % (epoch, n_epoch, time.time() - epoch_time, total_d_loss / n_iter,
                                                                            total_g_loss / n_iter)
    print(log)

    ## quick evaluation on train set
    if (epoch != 0) and (epoch % 10 == 0):
        out = sess.run(net_g_test.outputs, {t_image: sample_imgs_56})  #; print('gen sub-image:', out.shape, out.min(), out.max())
        print("[*] save images")
        tl.vis.save_images(out, [ni, ni], save_dir_gan + '/train_%d.png' % epoch)

    ## save model
    if (epoch != 0) and (epoch % 10 == 0):
        tl.files.save_npz(net_g.all_params, name=checkpoint_dir + '/g_{}.npz'.format("train"), sess=sess)
        tl.files.save_npz(net_d.all_params, name=checkpoint_dir + '/d_{}.npz'.format("train"), sess=sess)



 ** init lr: 0.000100  decay_every_init: 25, lr_decay: 0.000000 (for GAN)
Epoch [ 0/1000]    0 time: 23.2213s, d_loss: 2.02643013 g_loss: 0.07027414 (mse: 0.030769 vgg: 0.033019 adv: 0.006485)
Epoch [ 0/1000]    1 time: 2.8893s, d_loss: 3.20976257 g_loss: 0.07373558 (mse: 0.038615 vgg: 0.030571 adv: 0.004550)
Epoch [ 0/1000]    2 time: 2.8836s, d_loss: 3.14896226 g_loss: 0.06294714 (mse: 0.029146 vgg: 0.031610 adv: 0.002190)
Epoch [ 0/1000]    3 time: 2.8783s, d_loss: 2.46014833 g_loss: 0.07489321 (mse: 0.032075 vgg: 0.039581 adv: 0.003237)
Epoch [ 0/1000]    4 time: 2.8953s, d_loss: 1.87300801 g_loss: 0.05651864 (mse: 0.024050 vgg: 0.029824 adv: 0.002645)
Epoch [ 0/1000]    5 time: 2.9004s, d_loss: 2.49380851 g_loss: 0.05762159 (mse: 0.019912 vgg: 0.036958 adv: 0.000752)
Epoch [ 0/1000]    6 time: 2.8897s, d_loss: 2.06549168 g_loss: 0.13754508 (mse: 0.098246 vgg: 0.038471 adv: 0.000829)
Epoch [ 0/1000]    7 time: 2.9063s, d_loss: 2.26766849 g_loss: 0.06764884 (mse: 0.030535 vgg: 0.035

KeyboardInterrupt: 